<a href="https://colab.research.google.com/github/tarunvignesh1/Deep_learning_Project1/blob/main/Facemask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Project Facemask Detection 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch


#Checking for availablilty of CUDA
train_on_gpu = torch.cuda.is_available()
use_cuda = train_on_gpu

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [4]:
!pip install split-folders tqdm
import os
import numpy as np
import torchvision
from torchvision import models, datasets, transforms
import matplotlib.pyplot as plt
import splitfolders


datapath = '/content/drive/MyDrive/DeepLearningFiles/Facemask_data'


splitfolders.ratio(datapath, output="\content\drive\MyDrive\DeepLearningFiles\Facemask_data\splited", seed=1337, ratio=(.8, .1, .1), group_prefix=None)


Newpath = '/content/drive/MyDrive/DeepLearningFiles/Facemask_data/splited/'

trainpath = os.path.join(Newpath,'train/')
valpath = os.path.join(Newpath,'val/')
testpath = os.path.join(Newpath,'test/')

Streaming output truncated to the last 5000 lines.
Copying files: 2554 files [20:30,  2.17 files/s]
Copying files: 2555 files [20:30,  2.27 files/s]
Copying files: 2556 files [20:31,  2.05 files/s]
Copying files: 2557 files [20:31,  2.26 files/s]
Copying files: 2558 files [20:32,  2.17 files/s]
Copying files: 2559 files [20:33,  1.76 files/s]
Copying files: 2560 files [20:33,  1.83 files/s]
Copying files: 2561 files [20:34,  1.77 files/s]
Copying files: 2562 files [20:34,  1.91 files/s]
Copying files: 2563 files [20:35,  1.90 files/s]
Copying files: 2564 files [20:35,  2.15 files/s]
Copying files: 2565 files [20:35,  2.40 files/s]
Copying files: 2566 files [20:37,  1.33 files/s]
Copying files: 2567 files [20:37,  1.51 files/s]
Copying files: 2568 files [20:38,  1.62 files/s]
Copying files: 2569 files [20:38,  1.63 files/s]
Copying files: 2570 files [20:39,  1.76 files/s]
Copying files: 2571 files [20:39,  1.84 files/s]
Copying files: 2572 files [20:40,  1.94 files/s]
Copying files: 257

In [22]:
%matplotlib inline

transform_train = transforms.Compose([transforms.Resize((224,224)),transforms.RandomHorizontalFlip(p=0.5),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

transform_val = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [23]:

Newpath = '/content/splited1'

trainpath = os.path.join(Newpath,'train/')
valpath = os.path.join(Newpath,'val/')
testpath = os.path.join(Newpath,'test/')

train_data = datasets.ImageFolder(trainpath, transform=transform_train)
val_data = datasets.ImageFolder(valpath,transform=transform_val)
test_data = datasets.ImageFolder(testpath,transform=transform_val)

In [24]:
# define dataloader parameters
batch_size = 20
num_workers=0

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)

In [25]:
loaders = {'train': train_loader, 'valid': val_loader, 'test': test_loader}

In [10]:
vgg16 = models.vgg16(pretrained = True)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [11]:
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [12]:
print(vgg16.classifier[6])

Linear(in_features=4096, out_features=1000, bias=True)


In [13]:

# Freeze training for all "features" layers
for param in vgg16.features.parameters():
    param.requires_grad = False

In [14]:
# creating unique FC for the output
import torch.nn as nn
last_layer = nn.Linear(vgg16.classifier[6].in_features,2)

vgg16.classifier[6]=last_layer
if train_on_gpu:
    vgg16.cuda()
    pass
print(vgg16.classifier[6].out_features)

2


Training

In [26]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
def get_optimizer_transfer(model):
    optimizer = torch.optim.Adam(model.classifier.parameters(), lr = 0.001)
    return optimizer

In [27]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        # set the module to training mode
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## TODO: find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))
            
            

        ######################    
        # validate the model #
        ######################
        # set the model to evaluation mode
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## TODO: update average validation loss 
            output = model(data)
            loss = criterion(output, target)
            valid_loss += ((1 / (batch_idx + 1)) * (loss.data.item() - valid_loss))

    
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))

        ## TODO: if the validation loss has decreased, save the model at the filepath stored in save_path
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss

        
        
    return model

In [28]:
vgg16 = train(20, loaders, vgg16, get_optimizer_transfer(vgg16),criterion, use_cuda, 'face_detectvgg16.pt')
vgg16.load_state_dict(torch.load('face_detectvgg16.pt'))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch: 1 	Training Loss: 0.694553 	Validation Loss: 0.745323
Validation loss decreased (inf --> 0.745323).  Saving model ...
Epoch: 2 	Training Loss: 0.758381 	Validation Loss: 1.518096
Epoch: 3 	Training Loss: 1.451334 	Validation Loss: 2.042130
Epoch: 4 	Training Loss: 0.817417 	Validation Loss: 1.672157
Epoch: 5 	Training Loss: 0.480872 	Validation Loss: 2.154346
Epoch: 6 	Training Loss: 0.641803 	Validation Loss: 2.874306
Epoch: 7 	Training Loss: 0.480918 	Validation Loss: 2.719654
Epoch: 8 	Training Loss: 1.173372 	Validation Loss: 3.396736
Epoch: 9 	Training Loss: 0.516395 	Validation Loss: 6.197280
Epoch: 10 	Training Loss: 0.958178 	Validation Loss: 4.378013
Epoch: 11 	Training Loss: 0.358073 	Validation Loss: 4.801533
Epoch: 12 	Training Loss: 0.645067 	Validation Loss: 8.081475
Epoch: 13 	Training Loss: 1.233065 	Validation Loss: 13.913377
Epoch: 14 	Training Loss: 0.811351 	Validation Loss: 14.142462
Epoch: 15 	Training Loss: 1.230277 	Validation Loss: 11.516628
Epoch: 16 	T

<All keys matched successfully>

In [29]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    # set the module to evaluation mode
    model.eval()

    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [31]:
test(loaders, vgg16, criterion, use_cuda)

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Test Loss: 0.490838


Test Accuracy: 98% (749/757)
